In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import srgan
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
partition_size = 2048
reduction_factor = 4

In [5]:
with tf.variable_scope('generator') as gen:
    generator = srgan.Model_Keras(partition_size // 4, training = True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
with tf.variable_scope('discriminator') as dis:
    discriminator = srgan.MultiScaleDiscriminator(
        partition_size, num_filters = 24, training = True
    )

In [8]:
x = tf.placeholder(tf.float32, (None, 1))
y = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, partition_size // reduction_factor)
partitioned_y = malaya_speech.tf_featurization.pad_and_partition(y, partition_size)

In [9]:
mse_loss = tf.keras.losses.MeanSquaredError()
mae_loss = tf.keras.losses.MeanAbsoluteError()

In [10]:
y_hat = generator.model(partitioned_x)
y_hat.set_shape((None, partition_size, 1))
p_hat = discriminator.model(y_hat)
p = discriminator.model(partitioned_y)

In [11]:
p

[<tf.Tensor 'model_1_1/leaky_re_lu/LeakyRelu:0' shape=(?, 2048, 24) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_1/LeakyRelu:0' shape=(?, 1024, 24) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_3/LeakyRelu:0' shape=(?, 512, 48) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_5/LeakyRelu:0' shape=(?, 256, 96) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_7/LeakyRelu:0' shape=(?, 128, 192) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_9/LeakyRelu:0' shape=(?, 64, 240) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_11/LeakyRelu:0' shape=(?, 32, 288) dtype=float32>,
 <tf.Tensor 'model_1_1/leaky_re_lu_13/LeakyRelu:0' shape=(?, 16, 336) dtype=float32>]

In [12]:
y_hat = generator.model(partitioned_x)
y_hat.set_shape((None, partition_size, 1))
p_hat = discriminator.model(y_hat)
p = discriminator.model(partitioned_y)
fm_loss, adv_loss = 0.0, 0.0
for i in range(len(p_hat)):
    fm_loss += mae_loss(tf.ones_like(p_hat[i]), p_hat[i])
fm_loss /= i + 1
adv_loss += 10 * fm_loss
fm_loss, adv_loss

(<tf.Tensor 'truediv:0' shape=() dtype=float32>,
 <tf.Tensor 'add_12:0' shape=() dtype=float32>)

In [13]:
y_hat = generator.model(partitioned_x)
y_hat.set_shape((None, partition_size, 1))
p_hat = discriminator.model(y_hat)
p = discriminator.model(partitioned_y)
real_loss = 0.0
fake_loss = 0.0
for i in range(len(p)):
    real_loss += mse_loss(tf.ones_like(p[i]), p[i])
    fake_loss += mse_loss(tf.zeros_like(p[i]), p[i])

real_loss /= i + 1
fake_loss /= i + 1
dis_loss = real_loss + fake_loss

In [14]:
adv_loss, dis_loss

(<tf.Tensor 'add_12:0' shape=() dtype=float32>,
 <tf.Tensor 'add_29:0' shape=() dtype=float32>)

In [15]:
tf.trainable_variables()

[<tf.Variable 'generator/conv1d/kernel:0' shape=(9, 1, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/p_re_lu/alpha:0' shape=(1, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_1/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_1/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization/gamma:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization/beta:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/p_re_lu_1/alpha:0' shape=(1, 1) dtype=float32>,
 <tf.Variable 'generator/conv1d_2/kernel:0' shape=(3, 256, 256) dtype=float32>,
 <tf.Variable 'generator/conv1d_2/bias:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization_1/gamma:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/batch_normalization_1/beta:0' shape=(256,) dtype=float32>,
 <tf.Variable 'generator/conv1d_3/kernel:0' shape=(3, 256, 256) dtype=flo

In [16]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [17]:
y_lr, sr_lr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 11025)
y_hr, sr_hr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 44100)

In [18]:
%%time

sess.run([adv_loss, dis_loss], feed_dict = {x: np.expand_dims(y_lr, -1), y: np.expand_dims(y_hr, -1)})

CPU times: user 1min 19s, sys: 4.76 s, total: 1min 23s
Wall time: 13.5 s


[8.802048, 1.38012]

In [20]:
# y_[1].shape

In [21]:
# y_.reshape((-1)).shape

In [ ]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 44100)

In [22]:
saver = tf.train.Saver()

In [23]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [24]:
!ls -lh test

total 75080
-rw-r--r--  1 huseinzolkepli  staff    77B Feb 26 00:30 checkpoint
-rw-r--r--  1 huseinzolkepli  staff    34M Feb 26 00:30 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    11K Feb 26 00:30 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff   2.5M Feb 26 00:30 model.ckpt.meta


In [25]:
!rm -rf test